In [6]:
from typing import Callable, Dict, List, Optional, Tuple
import json

# Type definitions
ProposalMetadata = bytes
HandlerStorage = Dict[str, bytes]

class ProposeParams:
    def __init__(self, from_addr: str, frozen_token: int, proposal_metadata: ProposalMetadata):
        self.from_addr = from_addr
        self.frozen_token = frozen_token
        self.proposal_metadata = proposal_metadata

class PHInput:
    def __init__(self, packed_argument: bytes, handler_storage: HandlerStorage, proposal_info: ProposeParams):
        self.packed_argument = packed_argument
        self.handler_storage = handler_storage
        self.proposal_info = proposal_info

class PHOutput:
    def __init__(self, operations: List, handler_storage: HandlerStorage, guardian: Optional[str]):
        self.operations = operations
        self.handler_storage = handler_storage
        self.guardian = guardian

# Type aliases
ProposalHandler = Callable[[PHInput], PHOutput]
HandlerCheck = Callable[[Tuple[bytes, HandlerStorage]], None]

class HandlerInfo:
    def __init__(self, code: ProposalHandler, handler_check: HandlerCheck, is_active: bool):
        self.code = code
        self.handler_check = handler_check
        self.is_active = is_active

class AddHandlerParam:
    def __init__(self, name: str, code: ProposalHandler, handler_check: HandlerCheck):
        self.name = name
        self.code = code
        self.handler_check = handler_check
        
    def serialize(self) -> str:
        # Since functions can't be serialized directly, we use their names instead
        serialized_data = {
            "name": self.name,
            "code": self.code.__name__,  # Storing the name of the function
            "handler_check": self.handler_check.__name__  # Storing the name of the function
        }
        return json.dumps(serialized_data, indent=2)

In [7]:
# Example implementation of a proposal handler function
def example_proposal_handler(ph_input: PHInput) -> PHOutput:
    # Dummy operations list
    operations = []
    # Dummy guardian address
    guardian = None
    return PHOutput(operations=operations, handler_storage=ph_input.handler_storage, guardian=guardian)

# Example implementation of a handler check function
def example_handler_check(data: Tuple[bytes, HandlerStorage]) -> None:
    # Dummy check logic
    pass

# Instantiating AddHandlerParam
add_handler_param = AddHandlerParam(
    name="example_handler",
    code=example_proposal_handler,
    handler_check=example_handler_check
)

In [10]:
add_handler_param.serialize().encode().hex()

'7b0a2020226e616d65223a20226578616d706c655f68616e646c6572222c0a202022636f6465223a20226578616d706c655f70726f706f73616c5f68616e646c6572222c0a20202268616e646c65725f636865636b223a20226578616d706c655f68616e646c65725f636865636b220a7d'

In [11]:
ceva=add_handler_param.serialize()
ceva

'{\n  "name": "example_handler",\n  "code": "example_proposal_handler",\n  "handler_check": "example_handler_check"\n}'

In [ ]:
import json
from typing import Any, Dict, List, Optional, Callable, Tuple

# Type definitions
ProposalMetadata = bytes
HandlerStorage = Dict[str, bytes]

class ProposeParams:
    def __init__(self, from_addr: str, frozen_token: int, proposal_metadata: ProposalMetadata):
        self.from_addr = from_addr
        self.frozen_token = frozen_token
        self.proposal_metadata = proposal_metadata

    def to_dict(self) -> Dict[str, Any]:
        return {
            "from": self.from_addr,
            "frozen_token": self.frozen_token,
            "proposal_metadata": self.proposal_metadata.hex(),
        }

class PHInput:
    def __init__(self, packed_argument: bytes, handler_storage: HandlerStorage, proposal_info: ProposeParams):
        self.packed_argument = packed_argument
        self.handler_storage = handler_storage
        self.proposal_info = proposal_info

    def to_dict(self) -> Dict[str, Any]:
        return {
            "packed_argument": self.packed_argument.hex(),
            "handler_storage": {k: v.hex() for k, v in self.handler_storage.items()},
            "proposal_info": self.proposal_info.to_dict(),
        }

class PHOutput:
    def __init__(self, operations: List[Any], handler_storage: HandlerStorage, guardian: Optional[str]):
        self.operations = operations
        self.handler_storage = handler_storage
        self.guardian = guardian

    def to_dict(self) -> Dict[str, Any]:
        return {
            "operations": self.operations,
            "handler_storage": {k: v.hex() for k, v in self.handler_storage.items()},
            "guardian": self.guardian,
        }

# Type aliases
ProposalHandler = Callable[[PHInput], PHOutput]
HandlerCheck = Callable[[Tuple[bytes, HandlerStorage]], None]

class AddHandlerParam:
    def __init__(self, name: str, code: ProposalHandler, handler_check: HandlerCheck):
        self.name = name
        self.code = code
        self.handler_check = handler_check

    def serialize(self) -> str:
        serialized_data = {
            "name": self.name,
            # Note: we can't serialize the actual function, so here we're omitting code serialization
            # and focusing on parameters only.
            # In real-world usage, the function code would be deployed and referenced, not serialized.
        }
        return json.dumps(serialized_data, indent=2)

# Example instantiation
proposal_info = ProposeParams(from_addr="tz1...", frozen_token=1000, proposal_metadata=b'\x00\x01\x02')
ph_input = PHInput(packed_argument=b'\x01\x02\x03', handler_storage={"key1": b'value1'}, proposal_info=proposal_info)

add_handler_param = AddHandlerParam(
    name="example_handler",
    code=example_proposal_handler,  # In practice, the actual Michelson function code would be handled here.
    handler_check=None
)

# Serialize and print the output
serialized_output = add_handler_param.serialize()
print(serialized_output)


{
  "name": "example_handler",
  "code": {
    "packed_argument": "3d2901293ds",
    "handler_storage": {"string": "0210019231928dd123d2901293a3dafec9" },
    "proposal_info": {
      "from": "tz1T5kk65F9oZw2z1YV4osfcrX7eD5KtLj3c",
      "frozen_token": 10000,
      "proposal_metadata": "210019231928dd123d1d19283128417242133721999001"
    },
    "operations": <operations_placeholder>"
  },
  "handler_check": "<Handler check placeholder>"
}

In [28]:
handler_proposal='''{
  prim: "Pair",
  args: [
    {
      prim: "Pair",
      args: [
        {
          prim: "Lambda",
          args: [
            {
              prim: "Pair",
              args: [
                {
                  prim: "Pair",
                  args: [
                    { prim: "map", args: [{ prim: "string" }, { prim: "bytes" }] },
                    { prim: "bytes" }
                  ]
                },
                {
                  prim: "Pair",
                  args: [
                    { prim: "address" },
                    { prim: "nat" },
                    { prim: "bytes" }
                  ]
                }
              ]
            },
            {
              prim: "Pair",
              args: [
                {
                  prim: "Pair",
                  args: [
                    { prim: "option", args: [{ prim: "address" }] },
                    { prim: "map", args: [{ prim: "string" }, { prim: "bytes" }] }
                  ]
                },
                { prim: "list", args: [{ prim: "operation" }] }
              ]
            }
          ],
          annots: ["%code"]
        },
        {
          prim: "Lambda",
          args: [
            {
              prim: "Pair",
              args: [
                { prim: "bytes" },
                { prim: "map", args: [{ prim: "string" }, { prim: "bytes" }] }
              ]
            },
            { prim: "unit" }
          ],
          annots: ["%handler_check"]
        }
      ]
    },
    { string: "example_handler" }
  ]
}'''

In [32]:
import json
from typing import Dict, Any

def create_add_handler_proposal(handler_name: str, handler_code: str, handler_check: str) -> Dict[str, Any]:
    return {
        "prim": "Pair",
        "args": [
            {
                "prim": "Pair",
                "args": [
                    {
                        "prim": "Lambda",
                        "args": [
                            {
                                "prim": "Pair",
                                "args": [
                                    {
                                        "prim": "Pair",
                                        "args": [
                                            {"prim": "map", "args": [{"prim": "string"}, {"prim": "bytes"}]},
                                            {"prim": "bytes"}
                                        ]
                                    },
                                    {
                                        "prim": "Pair",
                                        "args": [
                                            {"prim": "address"},
                                            {"prim": "nat"},
                                            {"prim": "bytes"}
                                        ]
                                    }
                                ]
                            },
                            {
                                "prim": "Pair",
                                "args": [
                                    {
                                        "prim": "Pair",
                                        "args": [
                                            {"prim": "option", "args": [{"prim": "address"}]},
                                            {"prim": "map", "args": [{"prim": "string"}, {"prim": "bytes"}]}
                                        ]
                                    },
                                    {"prim": "list", "args": [{"prim": "operation"}]}
                                ]
                            }
                        ],
                        "annots": ["%code"]
                    },
                    {
                        "prim": "Lambda",
                        "args": [
                            {
                                "prim": "Pair",
                                "args": [
                                    {"prim": "bytes"},
                                    {"prim": "map", "args": [{"prim": "string"}, {"prim": "bytes"}]}
                                ]
                            },
                            {"prim": "unit"}
                        ],
                        "annots": ["%handler_check"]
                    }
                ]
            },
            {"string": handler_name}
        ]
    }

def create_propose_call_data(proposer_address: str, frozen_token_amount: int, packed_proposal: bytes) -> Dict[str, Any]:
    return {
        "entrypoint": "propose",
        "value": {
            "prim": "Pair",
            "args": [
                {"string": proposer_address},
                {"int": str(frozen_token_amount)},
                {"bytes": packed_proposal.hex()}
            ]
        }
    }

# Placeholder function for packing data (you'd replace this with actual Tezos packing logic)
def pack_data(data: Dict[str, Any]) -> bytes:
    # This is a simplified placeholder. In reality, you'd use a Tezos library to properly pack the data.
    return json.dumps(data).encode('utf-8')

    # Step 1: Define the handler details


In [36]:
handler_name = "example_handler"
handler_code = """
{ UNPAIR ;
    UNPAIR ;
    PUSH string "example_key" ;
    PUSH bytes 0x01020304 ;
    DIG 3 ;
    SOME ;
    DIG 2 ;
    UPDATE ;
    PAIR ;
    PUSH (option address) None ;
    PAIR ;
    NIL operation ;
    PAIR }
"""
handler_check = "{ DROP ; UNIT }"

handler_proposal = create_add_handler_proposal(handler_name, handler_code, handler_check)

packed_proposal = pack_data(handler_proposal)

proposer_address = "tz2QkZn4ERS7sDd3xBKoyjRcPTP76e7WLHd2"
frozen_token_amount = 10000  # 1 tez worth of tokens, adjust as needed
propose_call_data = create_propose_call_data(proposer_address, frozen_token_amount, packed_proposal)

print(packed_proposal.hex())

7b227072696d223a202250616972222c202261726773223a205b7b227072696d223a202250616972222c202261726773223a205b7b227072696d223a20224c616d626461222c202261726773223a205b7b227072696d223a202250616972222c202261726773223a205b7b227072696d223a202250616972222c202261726773223a205b7b227072696d223a20226d6170222c202261726773223a205b7b227072696d223a2022737472696e67227d2c207b227072696d223a20226279746573227d5d7d2c207b227072696d223a20226279746573227d5d7d2c207b227072696d223a202250616972222c202261726773223a205b7b227072696d223a202261646472657373227d2c207b227072696d223a20226e6174227d2c207b227072696d223a20226279746573227d5d7d5d7d2c207b227072696d223a202250616972222c202261726773223a205b7b227072696d223a202250616972222c202261726773223a205b7b227072696d223a20226f7074696f6e222c202261726773223a205b7b227072696d223a202261646472657373227d5d7d2c207b227072696d223a20226d6170222c202261726773223a205b7b227072696d223a2022737472696e67227d2c207b227072696d223a20226279746573227d5d7d5d7d2c207b227072696d223a20226c697374222c20226172677322

In [27]:
hex_string

'7b0a20207072696d3a202250616972222c0a2020617267733a205b0a202020207b0a2020202020207072696d3a202250616972222c0a202020202020617267733a205b0a20202020202020207b0a202020202020202020207072696d3a20224c616d626461222c0a20202020202020202020617267733a205b0a2020202020202020202020207b0a20202020202020202020202020207072696d3a202250616972222c0a2020202020202020202020202020617267733a205b0a202020202020202020202020202020207b0a2020202020202020202020202020202020207072696d3a202250616972222c0a202020202020202020202020202020202020617267733a205b0a20202020202020202020202020202020202020207b207072696d3a20226d6170222c20617267733a205b7b207072696d3a2022737472696e6722207d2c207b207072696d3a2022627974657322207d5d207d2c0a20202020202020202020202020202020202020207b207072696d3a2022627974657322207d0a2020202020202020202020202020202020205d0a202020202020202020202020202020207d2c0a202020202020202020202020202020207b0a2020202020202020202020202020202020207072696d3a202250616972222c0a202020202020202020202020202020202020617267733a205b0a2

In [59]:
import json

def create_add_handler_proposal(handler_name, code, handler_check):
    return {
        "prim": "Pair",
        "args": [
            {
                "prim": "Pair",
                "args": [
                    {
                        "prim": "Lambda",
                        "args": [
                            {
                                "prim": "Pair",
                                "args": [
                                    {
                                        "prim": "Pair",
                                        "args": [
                                            {"prim": "map", "args": [{"prim": "string"}, {"prim": "bytes"}]},
                                            {"prim": "bytes"}
                                        ]
                                    },
                                    {
                                        "prim": "Pair",
                                        "args": [
                                            {"prim": "address"},
                                            {"prim": "nat"},
                                            {"prim": "bytes"}
                                        ]
                                    }
                                ]
                            },
                            {
                                "prim": "Pair",
                                "args": [
                                    {
                                        "prim": "Pair",
                                        "args": [
                                            {"prim": "option", "args": [{"prim": "address"}]},
                                            {"prim": "map", "args": [{"prim": "string"}, {"prim": "bytes"}]}
                                        ]
                                    },
                                    {"prim": "list", "args": [{"prim": "operation"}]}
                                ]
                            }
                        ],
                        "code": code
                    },
                    {
                        "prim": "Lambda",
                        "args": [
                            {
                                "prim": "Pair",
                                "args": [
                                    {"prim": "bytes"},
                                    {"prim": "map", "args": [{"prim": "string"}, {"prim": "bytes"}]}
                                ]
                            },
                            {"prim": "unit"}
                        ],
                        "code": handler_check
                    }
                ]
            },
            {"string": handler_name}
        ]
    }

def create_propose_call_data(proposer_address, frozen_token_amount, proposal):
    return {
        "entrypoint": "propose",
        "value": {
            "prim": "Pair",
            "args": [
                {"string": proposer_address},
                {"int": str(frozen_token_amount)},
                proposal
            ]
        }
    }

# Simple Michelson code for the handler
handler_code = [
    {"prim": "DROP"},
    {"prim": "PUSH", "args": [{"prim": "string"}, {"string": "Hello, World!"}]},
    {"prim": "PUSH", "args": [{"prim": "bytes"}, {"bytes": "0123"}]},
    {"prim": "PUSH", "args": [{"prim": "string"}, {"string": "example_key"}]},
    {"prim": "DIG", "args": [{"int": "2"}]},
    {"prim": "PUSH", "args": [{"prim": "map", "args": [{"prim": "string"}, {"prim": "bytes"}]}, []]},
    {"prim": "PUSH", "args": [{"prim": "option", "args": [{"prim": "address"}]}, {"prim": "None"}]},
    {"prim": "PAIR"},
    {"prim": "PAIR"},
    {"prim": "NIL", "args": [{"prim": "operation"}]},
    {"prim": "PAIR"}
]

# Simple Michelson code for the handler check
handler_check_code = [
    {"prim": "DROP"},
    {"prim": "UNIT"}
]

# Example usage
handler_name = "example_handler"

proposal = create_add_handler_proposal(handler_name, handler_code, handler_check_code)
call_data = create_propose_call_data("tz1T5kk65F9oZw2z1YV4osfcrX7eD5KtLj3c", 10000, proposal)

print(bytes(str(call_data),"utf-8").hex())

7b27656e747279706f696e74273a202770726f706f7365272c202776616c7565273a207b277072696d273a202750616972272c202761726773273a205b7b27737472696e67273a2027747a32516b5a6e34455253377344643378424b6f796a526350545037366537574c486432277d2c207b27696e74273a20273130303030277d2c207b277072696d273a202750616972272c202761726773273a205b7b277072696d273a202750616972272c202761726773273a205b7b277072696d273a20274c616d626461272c202761726773273a205b7b277072696d273a202750616972272c202761726773273a205b7b277072696d273a202750616972272c202761726773273a205b7b277072696d273a20276d6170272c202761726773273a205b7b277072696d273a2027737472696e67277d2c207b277072696d273a20276279746573277d5d7d2c207b277072696d273a20276279746573277d5d7d2c207b277072696d273a202750616972272c202761726773273a205b7b277072696d273a202761646472657373277d2c207b277072696d273a20276e6174277d2c207b277072696d273a20276279746573277d5d7d5d7d2c207b277072696d273a202750616972272c202761726773273a205b7b277072696d273a202750616972272c202761726773273a205b7b277072696d273a20276f

'7b27656e747279706f696e74273a202770726f706f7365272c202776616c7565273a207b277072696d273a202750616972272c202761726773273a205b7b27737472696e67273a2027747a32516b5a6e34455253377344643378424b6f796a526350545037366537574c486432277d2c207b27696e74273a20273130303030277d2c207b277072696d273a202750616972272c202761726773273a205b7b277072696d273a202750616972272c202761726773273a205b7b277072696d273a20274c616d626461272c202761726773273a205b7b277072696d273a202750616972272c202761726773273a205b7b277072696d273a202750616972272c202761726773273a205b7b277072696d273a20276d6170272c202761726773273a205b7b277072696d273a2027737472696e67277d2c207b277072696d273a20276279746573277d5d7d2c207b277072696d273a20276279746573277d5d7d2c207b277072696d273a202750616972272c202761726773273a205b7b277072696d273a202761646472657373277d2c207b277072696d273a20276e6174277d2c207b277072696d273a20276279746573277d5d7d5d7d2c207b277072696d273a202750616972272c202761726773273a205b7b277072696d273a202750616972272c202761726773273a205b7b277072696d273a20276